# README

# Imports

In [1]:
import pandas as pd

# Introductory data description

## Top K most common

In [4]:
df = pd.read_csv('ipr_set.csv', delimiter= ';')

In [7]:
df

,Graph,Graph - Ont check,len(Graph - Ont check),Ont - Graph check,len(Ont - Graph check),Graph set lenght,Ont set lenght
0,http://example.com/dbpedia_3-5,"{'http://xmlns.com/foaf/0.1/givenName', 'http:...",15,"{'http://dbpedia.org/ontology/manxName', 'http...",245,1044,1274
1,http://example.com/dbpedia_3-5-1,"{'http://xmlns.com/foaf/0.1/givenName', 'http:...",15,"{'http://dbpedia.org/ontology/manxName', 'http...",242,1049,1276
2,http://example.com/dbpedia_2015-04,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",39,"{'http://dbpedia.org/ontology/enginePower', 'h...",1483,1375,2819
3,http://example.com/dbpedia_2015-10,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",38,"{'http://dbpedia.org/ontology/enginePower', 'h...",1490,1381,2833
4,http://example.com/dbpedia_2016-04,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",37,"{'http://dbpedia.org/ontology/enginePower', 'h...",1495,1391,2849
5,http://example.com/dbpedia_2016-10,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",39,"{'http://dbpedia.org/ontology/enginePower', 'h...",1510,1394,2865
6,http://example.com/dbpedia_3-6,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",18,"{'http://dbpedia.org/ontology/manxName', 'http...",246,1107,1335
7,http://example.com/dbpedia_3-7,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",24,"{'http://dbpedia.org/ontology/manxName', 'http...",363,1304,1643
8,http://example.com/dbpedia_3-8,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",22,"{'http://dbpedia.org/ontology/manxName', 'http...",476,1321,1775
9,http://example.com/dbpedia_3-9,"{'http://xmlns.com/foaf/0.1/thumbnail', 'http:...",24,"{'http://dbpedia.org/ontology/manxName', 'http...",975,1382,2333


# Structural measures of evolution

## Growth ratio

## Density

# Measuring evolution of content

## Knowledge degree

## Change ratio

## Vocabulary

### Vocabulary set

### Vocabulary uniqueness

### Vocabulary dynamicity

# Measuring quality between versions

## Instantiated class ratio

## Instantiated property ratio

## Inverse multiple inheritance number